In [ ]:
// using nightly-build
//#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:SandDance.InteractiveExtension, 1.0.53"
//#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22371.2 Microsoft.ML, 2.0.0-preview.22356.1 SandDance.InteractiveExtension, 1.0.53

# Exploring your own data - NFL Draft history from 2015 to 2021

OK, Let's do some using statements so that we have tools to work with our data.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;
using System.IO;

Do our files exist? They were downloaded from [pro-football-reference.com](https://www.pro-football-reference.com/), such as this page for the [2015 Draft](https://www.pro-football-reference.com/years/2015/draft.htm).

In [ ]:
var fileExists = File.Exists("drafts/2015Draft.csv");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

This file exists.  Nothing has blown up.....yet.

OK.  Make separate data frame objects for each draft from 2015 to 2021. Also, show the 2015 draft, to make sure the data makes sense.

In [ ]:
var draft2015 = DataFrame.LoadCsv("drafts/2015Draft.csv");
var draft2016 = DataFrame.LoadCsv("drafts/2016Draft.csv");
var draft2017 = DataFrame.LoadCsv("drafts/2017Draft.csv");
var draft2018 = DataFrame.LoadCsv("drafts/2018Draft.csv");
var draft2019 = DataFrame.LoadCsv("drafts/2019Draft.csv");
var draft2020 = DataFrame.LoadCsv("drafts/2020Draft.csv");
var draft2021 = DataFrame.LoadCsv("drafts/2021Draft.csv");
draft2021.Sample(15)

index,Rnd,Pick,Tm,Player,PlayerId,Pos,Age,To,AP1,PB,St,wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk,College/Univ,Column29
0,6,220,GNB,Isaiah McDuffie,McDuIs00,LB,22,2021,0,0,0,1,1,13,0,0,0,0,0,0,0,0,0,0,0,<null>,<null>,,Boston Col.,
1,3,100,TEN,Elijah Molden,MoldEl00,DB,22,2021,0,0,0,3,3,16,0,0,0,0,0,0,0,0,0,0,0,43,1,,Washington,
2,5,169,CLE,Richard LeCounte,LeCoRi00,DB,22,2021,0,0,0,1,1,9,0,0,0,0,0,0,0,0,0,0,0,2,<null>,,Georgia,
3,2,37,PHI,Landon Dickerson,DickLa00,OL,22,2021,0,0,1,6,6,14,0,0,0,0,0,0,0,0,0,0,0,<null>,<null>,,Alabama,
4,3,91,CLE,Anthony Schwartz,SchwAn00,WR,21,2021,0,0,0,1,1,14,0,0,0,0,0,6,39,0,10,135,1,2,<null>,,Auburn,
5,3,68,ATL,Jalen Mayfield,MayfJa00,OL,21,2021,0,0,1,7,7,16,0,0,0,0,0,0,0,0,0,0,0,<null>,<null>,,Michigan,
6,3,85,GNB,Amari Rodgers,RodgAm00,WR,21,2021,0,0,0,1,1,16,0,0,0,0,0,1,11,0,4,45,0,3,<null>,,Clemson,
7,7,256,GNB,Kylin Hill,HillKy00,RB,23,2021,0,0,0,0,0,8,0,0,0,0,0,10,24,0,1,5,0,1,<null>,,Mississippi St.,
8,4,112,DET,Amon-Ra St. Brown,StxxAm00,WR,21,2021,0,0,0,8,8,17,0,0,0,0,0,7,61,1,90,912,5,1,<null>,,USC,
9,5,183,ATL,Avery Williams,WillAv02,CB,23,2021,0,0,0,1,1,15,0,0,0,0,0,0,0,0,0,0,0,11,<null>,,Boise St.,


Hmmmmm...... the dataFrame has null values (and a blank column named Column29). Let's update some of this information.

- For "To", let's assume the year they were drafted was their last year. That means we need a DraftYear column.
- For null age, let's guess the median age of all the other draftees, rounded down.
- "0" values for all other columns seem fine.

First, Add the DraftYear and make it the year of that draft.

In [ ]:
int currentYear = 2022;
draft2021.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2021.Rows.Count()));
draft2021.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2021.Rows.Count()));
draft2021["DraftYear"].FillNulls(2021, true);
draft2021["To"].FillNulls(2021, true);
draft2021["YearsSinceDraft"].FillNulls(currentYear - 2021, true);

draft2020.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2020.Rows.Count()));
draft2020.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2020.Rows.Count()));
draft2020["DraftYear"].FillNulls(2020, true);
draft2020["To"].FillNulls(2020, true);
draft2020["YearsSinceDraft"].FillNulls(currentYear - 2020, true);

draft2019.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2019.Rows.Count()));
draft2019.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2019.Rows.Count()));
draft2019["DraftYear"].FillNulls(2019, true);
draft2019["To"].FillNulls(2019, true);
draft2019["YearsSinceDraft"].FillNulls(currentYear - 2019, true);

draft2018.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2018.Rows.Count()));
draft2018.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2018.Rows.Count()));
draft2018["DraftYear"].FillNulls(2018, true);
draft2018["To"].FillNulls(2018, true);
draft2018["YearsSinceDraft"].FillNulls(currentYear - 2018, true);

draft2017.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2017.Rows.Count()));
draft2017.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2017.Rows.Count()));
draft2017["DraftYear"].FillNulls(2017, true);
draft2017["To"].FillNulls(2017, true);
draft2017["YearsSinceDraft"].FillNulls(currentYear - 2017, true);

draft2016.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2016.Rows.Count()));
draft2016.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2016.Rows.Count()));
draft2016["DraftYear"].FillNulls(2016, true);
draft2016["To"].FillNulls(2016, true);
draft2016["YearsSinceDraft"].FillNulls(currentYear - 2016, true);

draft2015.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2015.Rows.Count()));
draft2015.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2015.Rows.Count()));
draft2015["DraftYear"].FillNulls(2015, true);
draft2015["To"].FillNulls(2015, true);
draft2015["YearsSinceDraft"].FillNulls(currentYear - 2015, true);

draft2021

index,Rnd,Pick,Tm,Player,PlayerId,Pos,Age,To,AP1,PB,St,wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk,College/Univ,Column29,DraftYear,YearsSinceDraft
0,1,1,JAX,Trevor Lawrence,LawrTr00,QB,21,2021,0,0,1,7,7,17,359,602,3641,12,17,73,334,2,0,0,0,2,<null>,,Clemson,,2021,1
1,1,2,NYJ,Zach Wilson,WilsZa00,QB,22,2021,0,0,1,5,5,13,213,383,2334,9,11,29,185,4,0,0,0,<null>,<null>,,BYU,,2021,1
2,1,3,SFO,Trey Lance,LancTr00,QB,21,2021,0,0,0,3,3,6,41,71,603,5,2,38,168,1,0,0,0,<null>,<null>,,North Dakota St.,,2021,1
3,1,4,ATL,Kyle Pitts,PittKy00,TE,20,2021,0,1,1,8,8,17,0,0,0,0,0,0,0,0,68,1026,1,3,<null>,,Florida,,2021,1
4,1,5,CIN,Ja'Marr Chase,ChasJa00,WR,21,2021,0,1,1,13,13,17,0,0,0,0,0,7,21,0,81,1455,13,2,<null>,,LSU,,2021,1
5,1,6,MIA,Jaylen Waddle,WaddJa00,WR,22,2021,0,0,1,7,7,16,0,0,0,0,0,2,3,1,104,1015,6,3,<null>,,Alabama,,2021,1
6,1,7,DET,Penei Sewell,SewePe00,OL,20,2021,0,0,1,7,7,16,0,0,0,0,0,0,0,0,0,0,0,<null>,<null>,,Oregon,,2021,1
7,1,8,CAR,Jaycee Horn,HornJa00,DB,21,2021,0,0,0,1,1,3,0,0,0,0,0,0,0,0,0,0,0,3,1,,South Carolina,,2021,1
8,1,9,DEN,Patrick Surtain II,SurtPa01,DB,21,2021,0,0,1,7,7,16,0,0,0,0,0,0,0,0,0,0,0,45,4,,Alabama,,2021,1
9,1,10,PHI,DeVonta Smith,SmitDe07,WR,22,2021,0,0,1,10,10,17,0,0,0,0,0,0,0,0,64,916,5,1,<null>,,Alabama,,2021,1


OK, let's get the median age of a drafted player (from 2021), then insert that age whenever we don't have that data.

In [ ]:
var medianAge = draft2021["Age"].Median();
int age = (int)medianAge;
draft2021["Age"].FillNulls(age, true);
draft2020["Age"].FillNulls(age, true);
draft2019["Age"].FillNulls(age, true);
draft2018["Age"].FillNulls(age, true);
draft2017["Age"].FillNulls(age, true);
draft2016["Age"].FillNulls(age, true);
draft2015["Age"].FillNulls(age, true);

Now that the ages are filled out, let's put all these drafts together into one big DataFrame.

In [ ]:
var drafts2015To2021 = draft2015.Clone();
DataFrame[] remainingDrafts = new DataFrame[] {draft2016, draft2017, draft2018, draft2019, draft2020, draft2021};
foreach (var draft in remainingDrafts)
{
    draft.Rows.ToList<DataFrameRow>()
        .ForEach(row => 
            {
                drafts2015To2021.Append(row, /*append in place*/ true);
            }
            );
}


// Remove Column29.
drafts2015To2021.Columns.Remove("Column29");

Console.Write("Number of rows in combined dataframe: ");
drafts2015To2021.Rows.Count()

Number of rows in combined dataframe: 

1786

All other null values can be zero.

Once we have all the data filled out, let's also list the career length for players who are already done playing (They have a "To" year before 2021).

For good measure, let's show the average number of games they player per year of their career.

In [ ]:
drafts2015To2021["Pos"].FillNulls("XX", true);

// Possible null rows are: wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk
drafts2015To2021["wAV"].FillNulls(0, true);
drafts2015To2021["DrAV"].FillNulls(0, true);
drafts2015To2021["G"].FillNulls(0, true);
drafts2015To2021["PassCmp"].FillNulls(0, true);
drafts2015To2021["PassAtt"].FillNulls(0, true);
drafts2015To2021["PassYds"].FillNulls(0, true);
drafts2015To2021["PassTD"].FillNulls(0, true);
drafts2015To2021["PassInt"].FillNulls(0, true);
drafts2015To2021["RushAtt"].FillNulls(0, true);
drafts2015To2021["RushYds"].FillNulls(0, true);
drafts2015To2021["RushTD"].FillNulls(0, true);
drafts2015To2021["Rec"].FillNulls(0, true);
drafts2015To2021["RecYds"].FillNulls(0, true);
drafts2015To2021["RecTD"].FillNulls(0, true);
drafts2015To2021["SoloTak"].FillNulls(0, true);
drafts2015To2021["Int"].FillNulls(0, true);
drafts2015To2021["Sk"].FillNulls("0", true);

// Create One More column showing years in career
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsInCareer", drafts2015To2021.Rows.Count()));
drafts2015To2021["YearsInCareer"] = (drafts2015To2021["To"] - drafts2015To2021["DraftYear"]) + 1;

// Average Games played per year
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<double>("GamesPerYearAvg", drafts2015To2021.Rows.Count()));
drafts2015To2021["GamesPerYearAvg"] = (drafts2015To2021["G"] / drafts2015To2021["YearsInCareer"]);

// Average Value per year of career
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<double>("ValuePerYear", drafts2015To2021.Rows.Count()));
drafts2015To2021["ValuePerYear"] = (drafts2015To2021["wAV"] / drafts2015To2021["YearsInCareer"]);



DataFrame.WriteCsv(drafts2015To2021,"drafts2015To2021.csv",',');

drafts2015To2021
//draft2021.ToTabularDataResource().ExploreWithSandDance().Display()

index,Rnd,Pick,Tm,Player,PlayerId,Pos,Age,To,AP1,PB,St,wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk,College/Univ,DraftYear,YearsSinceDraft,YearsInCareer,GamesPerYearAvg,ValuePerYear
0,1,1,TAM,Jameis Winston,WinsJa00,QB,21,2021,0,1,5,58,54,83,1665,2720,20982,135,91,288,1204,11,0,0,0,0,0,0,Florida St.,2015,7,7,11.857142857142858,8.285714285714286
1,1,2,TEN,Marcus Mariota,MariMa01,QB,21,2021,0,0,4,45,44,74,1128,1795,13437,77,45,264,1574,13,2,62,1,0,0,0,Oregon,2015,7,7,10.571428571428571,6.428571428571429
2,1,3,JAX,Dante Fowler,FowlDa00,OLB,21,2021,0,0,2,23,5,91,0,0,0,0,0,0,0,0,0,0,0,137,0,35,Florida,2015,7,7,13,3.2857142857142856
3,1,4,OAK,Amari Cooper,CoopAm00,WR,21,2021,0,4,7,50,24,108,0,0,0,0,0,14,41,0,517,7076,46,0,0,0,Alabama,2015,7,7,15.428571428571429,7.142857142857143
4,1,5,WAS,Brandon Scherff,ScheBr00,T,23,2021,1,5,7,47,47,89,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Iowa,2015,7,7,12.714285714285714,6.714285714285714
5,1,6,NYJ,Leonard Williams,WillLe02,DE,21,2021,0,1,6,49,32,112,0,0,0,0,0,0,0,0,0,0,0,198,1,35.5,USC,2015,7,7,16,7
6,1,7,CHI,Kevin White,WhitKe00,WR,23,2021,0,0,0,2,2,23,0,0,0,0,0,1,9,0,26,323,0,1,0,0,West Virginia,2015,7,7,3.2857142857142856,0.2857142857142857
7,1,8,ATL,Vic Beasley,BeasVi00,OLB,23,2020,1,1,5,32,32,88,0,0,0,0,0,0,0,0,0,0,0,122,1,37.5,Clemson,2015,7,6,14.666666666666666,5.333333333333333
8,1,9,NYG,Ereck Flowers,FlowEr00,T,21,2021,0,0,7,35,18,105,0,0,0,0,0,0,0,0,1,-11,0,0,0,0,Miami (FL),2015,7,7,15,5
9,1,10,STL,Todd Gurley,GurlTo01,RB,21,2020,2,3,6,54,50,88,0,0,0,0,0,1460,6082,67,243,2254,12,0,0,0,Georgia,2015,7,6,14.666666666666666,9
